# Understanding the problem

## Load the data

In [1]:
#pip install opendatasets --upgrade
#import opendatasets as od
#od.download('https://www.kaggle.com/competitions/santa-2023/data')

#pip install sympy

In [1]:
import numpy as np
import pandas as pd
from ast import literal_eval
from pathlib import Path
from pprint import pprint
from sympy.combinatorics import Permutation

In [2]:
data_dir = Path("santa-2023")

puzzle_info = pd.read_csv(data_dir / 'puzzle_info.csv', index_col='puzzle_type')
# Parse allowed_moves
puzzle_info['allowed_moves'] = puzzle_info['allowed_moves'].apply(literal_eval)

puzzles = pd.read_csv(data_dir / 'puzzles.csv', index_col='id')
# Parse color states
puzzles = puzzles.assign(
    initial_state=lambda df: df['initial_state'].str.split(';'),
    solution_state=lambda df: df['solution_state'].str.split(';')
)

sample_submission = pd.read_csv(data_dir / 'sample_submission.csv', index_col='id')

In [4]:
# Filter cube
cube_index = puzzle_info.index.str.contains('cube')
puzzle_info_cube = puzzle_info[cube_index]

puzzles_cube = puzzles.loc[puzzles['puzzle_type'].str.contains('cube')]

In [7]:
def cube_state_to_faces(state):
    """Convert a state list to a dictionary of labeled faces."""
    n = int(np.sqrt(len(state) / 6))  # cube_n/n/n
    n2 = n ** 2
    labels = f"d{n-1},f0,r0,f{n-1},r{n-1},d0".split(',')
    faces = {}
    for i, l in enumerate(labels):
        face = state[n2 * i : n2 * (i + 1)]
        faces[l] = np.asarray(face).reshape(n, n).tolist()
    return faces

In [6]:
puzzles_cube

,puzzle_type,solution_state,initial_state,num_wildcards
id,,,,
0,cube_2/2/2,"[A, A, A, A, B, B, B, B, C, C, C, C, D, D, D, ...","[D, E, D, A, E, B, A, B, C, A, C, A, D, C, D, ...",0
1,cube_2/2/2,"[A, A, A, A, B, B, B, B, C, C, C, C, D, D, D, ...","[D, E, C, B, B, E, F, A, F, D, B, F, F, E, B, ...",0
2,cube_2/2/2,"[A, A, A, A, B, B, B, B, C, C, C, C, D, D, D, ...","[E, F, C, C, F, A, D, D, B, B, A, F, E, B, C, ...",0
3,cube_2/2/2,"[A, A, A, A, B, B, B, B, C, C, C, C, D, D, D, ...","[A, C, E, C, F, D, E, D, A, A, F, A, B, D, B, ...",0
4,cube_2/2/2,"[A, A, A, A, B, B, B, B, C, C, C, C, D, D, D, ...","[E, D, E, D, A, E, F, B, A, C, F, D, F, D, C, ...",0
...,...,...,...,...
279,cube_19/19/19,"[A, A, A, A, A, A, A, A, A, A, A, A, A, A, A, ...","[B, D, F, E, E, F, C, C, E, F, C, E, F, C, A, ...",0
280,cube_19/19/19,"[A, A, A, A, A, A, A, A, A, A, A, A, A, A, A, ...","[D, E, E, D, E, B, E, E, A, C, F, A, F, B, F, ...",0
281,cube_33/33/33,"[A, A, A, A, A, A, A, A, A, A, A, A, A, A, A, ...","[B, F, C, B, E, F, A, F, F, C, B, A, F, A, D, ...",0
